# TOOL MCGVYERING: TOOL CREATION WITH AUXILARY OBJECTS


## Required Libraries & Parameters

In [ ]:
import sys
sys.path.append('../src')
import numpy as np
import robotic as ry

from RAI import RAI
from SEG import SEG
from SQF import SQF
from Score import Score

cam_list = ["cam_front", "cam_back", "cam_left", "cam_right", "cam_up", "cam_down"]
filter = 1
model_path = "../src/models/tools/simple/parts"
target_path = "../src/point_clouds_ref/"
scene = ry.Config()
scene.addFile("../src/config/tools_simple1_s.g")
scene.view(False)

reference_tool = "shovel" # hammer, rake, scraper, axe # Select a reference tool

## STEP 1: Generate the point cloud for the reference tool

In [ ]:
print("Generating the reference point cloud")
rai = RAI(verbose=0)
segments_ref = rai.generate_ref_point_cloud(model_path, cam_list, filter = filter, object_name=reference_tool)

## STEP 2: Generate the point cloud for the candidate objects

In [ ]:
print("Generating the candidate point cloud")
cam_list = ["cam_front_left", "cam_front_right", "cam_back_right", "cam_back_left", "cam_up", "cam_front", "cam_left", "cam_back"] 
ptc_cand, _, _ = rai.get_raw_point_cloud(scene, cam_list, filter = filter)
   

## STEP 3: Remove the plane from the scene

In [ ]:
print("Removing the plane from the scene")
seg = SEG(verbose=0)
pcl_filtered = seg.RANSAC_plane(ptc_cand)
segments_cand = seg.segment_objects(pcl_filtered, scene)

## STEP 4: Fit SuperQuadric function to the objects

In [ ]:
print("Fitting SuperQuadric function to the objects")
param_ref  = {}
for key, pcd in segments_ref.items():
    sqf = SQF(pcd, verbose=0)
    param, _, _, _ = sqf.fit(sq_type=[0])
    if len(param) < 12:
        param = np.pad(param, (0, 12 - len(param)), mode='constant', constant_values=0)
    param_ref[key] = param

param_cand = {}
for key, pcd in segments_cand.items():
    sqf = SQF(pcd, verbose=0)
    param, _, _, _ = sqf.fit(sq_type=[0])
    if len(param) < 12:
        param = np.pad(param, (0, 12 - len(param)), mode='constant', constant_values=0)
    param_cand[key] = param

## STEP 5: Find the best matching candidates objects to the reference tool set

In [ ]:
print("Finding the best matching candidate objects to the reference tool set")
score = Score(param_ref, param_cand, verbose=1)
match_1, match_2 = score.algorithm_1()

## STEP 6: Create the tool

In [ ]:
rai.create_tool(scene, match_1, match_2)